In [ ]:
import pandas as pd
import requests
import time

airportsurl = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
airportscols = [
    'airport_id', 'name', 'city', 'country', 'iata', 'icao',
    'latitude', 'longitude', 'altitude', 'timezone',
    'dst', 'tz_database_time_zone', 'type', 'source'
]

airports = pd.read_csv(airportsurl, header=None, names=airportscols)
airports = airports[airports['country'] == 'United States']

routesurl = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat'
routescols = [
    'airlines', 'airline_id', 'source_aiport', 'source_airport_id', 
    'destination_airport', 'destination_airport_id',
    'codeshare', 'stops', 'equipment'
]

routes = pd.read_csv(routesurl, header=None, names=routescols)

source_airports = set(routes['source_aiport'].unique())
dest_airports = set(routes['destination_airport'].unique())
active_iata_codes = source_airports.union(dest_airports)

# Filter the US airports list to only include those present in routes data
airports_us_active = airports[airports['iata'].isin(active_iata_codes)].copy()

# Drop airports with missing IATA codes or coordinates, as they are unusable
airports_us_active.dropna(subset=['iata', 'latitude', 'longitude', 'city', 'icao', 'timezone', 'altitude'], inplace=True)

NOAA_TOKEN = "qRQFzVEhbSEwKcrbayQhBBmxeiDfxPCc"

# Define date range for weather
START_DATE = "2024-01-01"
END_DATE = "2024-12-31"

In [ ]:
# Headers for all API requests
api_headers = {'token': NOAA_TOKEN}

def find_nearest_station(lat, lon):
    """Find the closest weather station to a given latitude and longitude."""
    # Search for stations within a 2-degree box around the airport
    search_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/stations"
    params = {
        "datasetid": "GHCND", # Daily Summaries
        "datacategoryid": "TEMP", # Ensure it has temperature data
        "extent": f"{lat-1},{lon-1},{lat+1},{lon+1}",
        "startdate": START_DATE,
        "enddate": END_DATE,
        "limit": 1 # Find the single closest one
    }
    try:
        response = requests.get(search_url, headers=api_headers, params=params)
        response.raise_for_status() # Raise an exception for bad status codes
        station_data = response.json()
        if station_data and 'results' in station_data and station_data['results']:
            return station_data['results'][0]['id']
    except requests.exceptions.RequestException as e:
        print(f"  - API Error finding station: {e}")
    return None

def get_weather_data(station_id):
    """Fetch daily weather data for a given station ID."""
    data_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    # TMAX: Max Temp, TMIN: Min Temp, PRCP: Precipitation, SNOW: Snowfall, AWND: Avg Wind Speed
    params = {
        "datasetid": "GHCND",
        "stationid": station_id,
        "startdate": START_DATE,
        "enddate": END_DATE,
        "limit": 1000, # Max limit per request
        "units": "metric", # Use metric for easier conversion
        "datatypeid": "TMAX,TMIN,PRCP,SNOW,AWND"
    }
    try:
        response = requests.get(data_url, headers=api_headers, params=params)
        response.raise_for_status()
        weather_data = response.json()
        if weather_data and 'results' in weather_data:
            return weather_data['results']
    except requests.exceptions.RequestException as e:
        print(f"  - API Error fetching weather: {e}")
    return None

In [16]:
weather_results = []
total_airports = len(airports_us_active)

for index, airport in airports_us_active.iterrows():
    iata = airport['iata']
    lat = airport['latitude']
    lon = airport['longitude']

    print(f"Processing ({index+1}/{total_airports}): {airport['name']} ({iata})")

    # Respect NOAA's 5 requests/second limit
    time.sleep(0.25)

    station_id = find_nearest_station(lat, lon)
    if not station_id:
        print(f"  - Could not find a suitable weather station for {iata}. Skipping.")
        continue

    print(f"  - Found nearest station: {station_id}")
    daily_weather = get_weather_data(station_id)

    if not daily_weather:
        print(f"  - Could not fetch weather data for station {station_id}. Skipping.")
        continue

    # Process and aggregate the data using pandas
    weather_df = pd.DataFrame(daily_weather)
    
    # Pivot the data to have datatypes as columns
    weather_pivot = weather_df.pivot_table(index='date', columns='datatype', values='value').reset_index()

    # Data cleaning and conversion
    if 'TMAX' in weather_pivot.columns:
        weather_pivot['TMAX'] /= 10.0 # Tenths of Celsius to Celsius
    if 'TMIN' in weather_pivot.columns:
        weather_pivot['TMIN'] /= 10.0 # Tenths of Celsius to Celsius
    if 'PRCP' in weather_pivot.columns:
        weather_pivot['PRCP'] /= 10.0 # Tenths of mm to mm
    if 'AWND' in weather_pivot.columns:
        weather_pivot['AWND'] /= 10.0 # Tenths of m/s to m/s

    # Calculate aggregated metrics
    summary = {
        'iata': iata,
        'latitude': lat,
        'longitude': lon,
        'NOAA_Station_ID': station_id,
        'Avg_Max_Temp_C': weather_pivot['TMAX'].mean() if 'TMAX' in weather_pivot.columns else None,
        'Avg_Min_Temp_C': weather_pivot['TMIN'].mean() if 'TMIN' in weather_pivot.columns else None,
        'Total_Precip_mm': weather_pivot['PRCP'].sum() if 'PRCP' in weather_pivot.columns else None,
        'Total_Snow_mm': weather_pivot['SNOW'].sum() if 'SNOW' in weather_pivot.columns else None,
        'Avg_Wind_Speed_ms': weather_pivot['AWND'].mean() if 'AWND' in weather_pivot.columns else None,
    }

    weather_results.append(summary)
    print(f"Successfully processed and aggregated weather for {iata}.")

final_df = pd.DataFrame(weather_results)

Processing (3213/552): Barter Island LRRS Airport (BTI)
  - Found nearest station: GHCND:USW00027401
Successfully processed and aggregated weather for BTI.
Processing (3215/552): Cape Lisburne LRRS Airport (LUR)
  - Found nearest station: GHCND:USC00507431
  - API Error fetching weather: 503 Server Error: Service Unavailable for url: https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND%3AUSC00507431&startdate=2024-01-01&enddate=2024-12-31&limit=1000&units=metric&datatypeid=TMAX%2CTMIN%2CPRCP%2CSNOW%2CAWND
  - Could not fetch weather data for station GHCND:USC00507431. Skipping.
Processing (3216/552): Point Lay LRRS Airport (PIZ)
  - Found nearest station: GHCND:USC00507442
Successfully processed and aggregated weather for PIZ.
Processing (3217/552): Hilo International Airport (ITO)
  - Found nearest station: GHCND:USC00513977
Successfully processed and aggregated weather for ITO.
Processing (3219/552): Bettles Airport (BTT)
  - Found nearest station: GHCND:USR0

In [ ]:
# Save data just in case
final_df.to_csv('airports.csv', index=False)
routes.to_csv('routes.csv', index=False)

In [37]:
print(final_df.head(20))
print(routes.head(20))

routes['source_aiport'] = routes['source_aiport'].str.strip()
routes.rename(columns={'source_aiport': 'source_airport'}, inplace=True)

   iata   latitude   longitude    NOAA_Station_ID  Avg_Max_Temp_C  \
0   BTI  70.134003 -143.582001  GHCND:USW00027401        0.045455   
1   PIZ  69.732903 -163.005005  GHCND:USC00507442       -0.429478   
2   ITO  19.721399 -155.048004  GHCND:USC00513977        2.907433   
3   BTT  66.913902 -151.529007  GHCND:USR0000AKAN        0.099781   
4   FYU  66.571503 -145.250000  GHCND:USR0000ABCK        0.475078   
5   TLJ  62.894402 -155.977005  GHCND:USW00026510        0.637649   
6   SNP  57.167301 -170.220001  GHCND:USW00025628        0.344413   
7   STG  56.578300 -169.662003  GHCND:USW00025628        0.344413   
8   ILI  59.754398 -154.910996  GHCND:USC00507570        0.827726   
10  GFK  47.949299  -97.176102  GHCND:USC00210018        1.328960   
12  PRC  34.654499 -112.419998  GHCND:USC00020586        2.608163   
13  TTN  40.276699  -74.813499  GHCND:USC00076410        1.991291   
14  OAK  37.721298 -122.221001  GHCND:USC00040212        2.133033   
15  OMA  41.303200  -95.894096  GH

## Upload to neo4j Aura

In [ ]:
from neo4j import GraphDatabase

#Credentials
NEO4J_URI = "neo4j+s://23017e96.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = ""

In [ ]:
def upload_airports(driver, airports_data):
    """Uploads airport data as :Airport nodes."""
    print("--- Uploading Airports ---")
    
    # Use a session to run queries
    with driver.session() as session:
        # Create a uniqueness constraint for faster lookups and data integrity
        session.run("""
            CREATE CONSTRAINT airport_iata IF NOT EXISTS 
            FOR (a:Airport) REQUIRE a.iata IS UNIQUE
        """)
        print("   - Uniqueness constraint on :Airport(iata) ensured.")

        # Cypher query to unwind the list of airports and create nodes
        airport_query = """
        UNWIND $airports as airport
        MERGE (a:Airport {iata: airport.iata})
        SET a.latitude = airport.latitude,
            a.longitude = airport.longitude,
            a.avgMaxTempC = airport.Avg_Max_Temp_C,
            a.avgMinTempC = airport.Avg_Min_Temp_C,
            a.totalPrecipMM = airport.Total_Precip_mm,
            a.totalSnowMM = airport.Total_Snow_mm
        RETURN count(a) as node_count
        """
        result = session.run(airport_query, airports=airports_data)
        node_count = result.single()["node_count"]
        print(f"   - Merged {node_count} :Airport nodes.")

def upload_routes(driver, routes_data):
    """Uploads flight routes as [:FLIES_TO] relationships."""
    print("--- Uploading Routes ---")
    
    with driver.session() as session:
        # Cypher query to create relationships between existing airports
        route_query = """
        UNWIND $routes as route
        MATCH (origin:Airport {iata: route.source_airport})
        MATCH (destination:Airport {iata: route.destination_airport})
        MERGE (origin)-[r:FLIES_TO]->(destination)
        SET r.airline = route.airline,
            r.airline_id = route.airline_id,
            r.stops = route.stops,
            r.equipment = route.equipment
        RETURN count(r) as rel_count
        """
        result = session.run(route_query, routes=routes_data)
        rel_count = result.single()["rel_count"]
        print(f"   - Merged {rel_count} [:FLIES_TO] relationships.")

In [39]:
if __name__ == "__main__":
    # Convert DataFrames to a list of dictionaries for the driver
    airports_list = final_df.to_dict('records')
    routes_list = routes.to_dict('records')

    # Establish connection to the database
    try:
        driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        driver.verify_connectivity()
        print("Successfully connected to Neo4j Aura.")

        # Run the upload functions
        upload_airports(driver, airports_list)
        upload_routes(driver, routes_list)

        # Verify the upload by counting nodes and relationships
        print("--- Verifying Upload ---")
        with driver.session() as session:
            node_result = session.run("MATCH (n:Airport) RETURN count(n) as count").single()
            rel_result = session.run("MATCH ()-[r:FLIES_TO]->() RETURN count(r) as count").single()
            print(f"   - Total :Airport nodes in DB: {node_result['count']}")
            print(f"   - Total [:FLIES_TO] relationships in DB: {rel_result['count']}")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if 'driver' in locals() and driver:
            driver.close()
            print("Connection to Neo4j closed.")

Successfully connected to Neo4j Aura.
--- Uploading Airports ---
   - Uniqueness constraint on :Airport(iata) ensured.
   - Merged 421 :Airport nodes.
--- Uploading Routes ---
   - Merged 5853 [:FLIES_TO] relationships.
--- Verifying Upload ---
   - Total :Airport nodes in DB: 421
   - Total [:FLIES_TO] relationships in DB: 3100
Connection to Neo4j closed.
